# Общий план

## 1. Запустим Parler-TTS на тестовом примере (пример в ноутбуке results.ipynb).


Note: изначально непонятно, какая часть всей выборки использовалась для претрейна, который выложен на huggingface. Поэтому также будем сравнивать на LibriSpeech test clean для объективности.

Разделим датасет Jenny на тренировочный и тестовый наборы: выделим под тестовую часть ~0.3% (в итоге 63 случайных аудиозаписей из набора).

Также возьмем небольшую часть датасета LibriSpeech test clean (63 случайных аудиозаписей). (всё проделано в ноутбуке results.ipynb)

Все расчеты проводятся на одной T4 GPU (поэтому для бенчмарков используем немного записей).

## 2. Что входит в оценку синтеза речи

Нам важно учитывать характеристики:

- Естественность (Naturalness)

- Разборчивость (Intelligibility): Насколько четко произносятся слова.

- Сходство голосов: насколько синтезированный голос похож на целевой.

## 3. Метрики для оценки

Для каждой характеристики будем использовать следующие метрики:

- Естественность: MOS (Mean Opinion Score) — субъективная оценка от 1 до 5, где 5 — максимальная естественность (обычно оценивается людьми, что очень субьективно). Также есть разные Neural MOS, к примеру [UTMOS](https://arxiv.org/abs/2204.02152), будем использовать его.
- Разборчивость: WER (Word Error Rate) — процент ошибок в распознавании слов. Будем брать хороший ASR, полученное аудио транскрибировать и сравнивать с исходной транскрибацией.

- Сходство голосов: будем считать [Speaker Encoder Cosine Similarity - SECS](https://arxiv.org/abs/2104.05557), используя модель эмбеддингов WavLM

Почему именно эти метрики? Они являются наиболее объективными для TTS, хотя и тут вопросы: какой ASR использовать для WER, какую именно модель для Neural MOS лучше использовать, из-за этого во многих публикациях могут быть различия в оценках и не будет объективности.

Можно также было бы оценивать более субъективные вещи: вручную и/или попросить группу людей сделать оценку MOS, также насколько точно передается стиль тона в промпте Parler TTS, но это не сильно точно.


In [ ]:
!git clone https://github.com/Nikait/Parler_TTS_estimation
!mv Parler_TTS_estimation/compute_utmos.py ./compute_utmos.py
!mv Parler_TTS_estimation/compute_wer.py ./compute_wer.py
!mv Parler_TTS_estimation/compute_secs.py ./compute_secs.py

# пример запуска

In [1]:
!pip install git+https://github.com/huggingface/parler-tts.git

  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-pepp9cte
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-pepp9cte
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-ykq24jmo/descript-audiotools_cacd0e69259f451081dc6f5b2fb41f99
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-ykq24jmo/descript-audiotools_cacd0e69259f451081dc6f5b2fb41f99
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.9 MB/s eta 0:00:00
  Prep

In [2]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration

In [3]:
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-jenny-30H").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-jenny-30H")

prompt = "Hi! What you did? Are you mad?"
description = "A young female speaks normal, audio is very clear."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio = generation.cpu().numpy().squeeze()

print("sample rate: ", model.config.sampling_rate)
sf.write("out.wav", audio, model.config.sampling_rate)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_ra

sample rate:  44100


In [4]:
from IPython.display import Audio, display

display(Audio("out.wav", autoplay=True))

# Загружаем Jenny датасет, отделяем небольшую тестовую выборку

In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install datasets

In [10]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

dataset = load_dataset("reach-vb/jenny_tts_dataset")
data = dataset["train"] # в датасете только одна часть

In [11]:
# отделим небольшую тестовую выборку
data = data.train_test_split(test_size=0.003)

# функция для генерации одного аудио
def generate_audio(text, description, model, tokenizer, device):
    input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
    prompt_input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
    audio = generation.cpu().numpy().squeeze()
    return audio

In [12]:
import os
import soundfile as sf

# Создание директории 'test', если она не существует
if not os.path.exists('test'):
    os.makedirs('test')

# Открытие файла для сохранения текстов
with open("test/transcriptions.txt", "w", encoding="utf-8") as text_file:
    # Обработка каждого примера в тестовой выборке
    for i, example in enumerate(data['test']):
        text = example["transcription"]
        description = "A young female speaks normal, audio is very clear."  # Описание для генерации

        print(f"Predicting the {i:>5}/{len(data['test'])}")

        # Генерация аудио с помощью TTS
        audio = generate_audio(text, description, model, tokenizer, device)

        # Сохранение сгенерированного аудиофайла
        generated_audio_file = f"test/generated_audio_{i}.wav"
        sf.write(generated_audio_file, audio, model.config.sampling_rate)

        # Извлечение и сохранение исходного аудио
        original_audio = example["audio"]["array"]  # Аудиоданные
        original_sr = example["audio"]["sampling_rate"]  # Частота дискретизации
        original_audio_file = f"test/original_audio_{i}.wav"
        sf.write(original_audio_file, original_audio, original_sr)

        # Сохранение текста в файл
        text_file.write(f"generated_audio_{i}.wav: {text}\n")

print("All audio files and transcriptions have been saved.")

Predicting the     0/63
Predicting the     1/63
Predicting the     2/63
Predicting the     3/63
Predicting the     4/63
Predicting the     5/63
Predicting the     6/63
Predicting the     7/63
Predicting the     8/63
Predicting the     9/63
Predicting the    10/63
Predicting the    11/63
Predicting the    12/63
Predicting the    13/63
Predicting the    14/63
Predicting the    15/63
Predicting the    16/63
Predicting the    17/63
Predicting the    18/63
Predicting the    19/63
Predicting the    20/63
Predicting the    21/63
Predicting the    22/63
Predicting the    23/63
Predicting the    24/63
Predicting the    25/63
Predicting the    26/63
Predicting the    27/63
Predicting the    28/63
Predicting the    29/63
Predicting the    30/63
Predicting the    31/63
Predicting the    32/63
Predicting the    33/63
Predicting the    34/63
Predicting the    35/63
Predicting the    36/63
Predicting the    37/63
Predicting the    38/63
Predicting the    39/63
Predicting the    40/63
Predicting the  

# Оценка естественности (UTMOS) на Jenny

In [14]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!python3 compute_utmos.py

Downloading: "https://github.com/tarepan/SpeechMOS/zipball/v1.2.0" to /root/.cache/torch/hub/v1.2.0.zip
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Downloading: "https://github.com/tarepan/SpeechMOS/releases/download/v1.0.0/utmos22_strong_step7459_v1.pt" to /root/.cache/torch/hub/checkpoints/utmos22_strong_step7459_v1.pt
100% 392M/392M [00:15<00:00, 26.6MB/s]
File: test/generated_audio_0.wav
UTMOS: 4.3870
----------------------------------------------------------------------------------------------------
File: test/generated_audio_1.wav
UTMOS: 4.0984
----------------------------------------------------------------------------------------------------
File: test/generated_audio_2.wav
UTMOS: 3.5902
----------------------------------------------------------------------------------------------------
File: te

Среднее значение - 4.17 / 5, что показывает очень высокую естественность сгенерированных аудио

# Оценка Разборчивости, точность произношения (WER) на Jenny

Будем использовать Whisper в качестве ASR'а

In [15]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.2 MB/s eta 0:00:00


In [17]:
!python3 compute_wer.py

2025-01-07 22:45:34.198829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 22:45:34.219947: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 22:45:34.226061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 22:45:35.525214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
config.json: 100% 1.27k/1.27k [00:00<00:00, 5.92MB/s]
model.safetensors: 100% 3.09G/3.09G [01:12<00:00, 42.3MB/s]
generation_config.json: 100% 3.90k/3.90k [00:00<00:00, 21.3MB/s]
tokenizer_config.json: 100% 283k/283k [00:00<00:00, 665kB/s]
v

Средний WER довольно маленький, в основном ошибки происходят из-за различий в пунктуации

# Оценка сходства голосов (SECS) на Jenny

In [22]:
!python3 compute_secs.py

2025-01-07 23:01:04.819192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 23:01:04.851974: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 23:01:04.861895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 23:01:06.713816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
File: 0
Original: test/original_audio_0.w

довольно высокое SECS, для примера у Tacotroon в [исходной статье](https://arxiv.org/abs/2104.05557) около 0.75 на 55 аудио из VCTK датасета

# Распаковка датасета Libri Speech test clean

In [23]:
import os
import tarfile
import shutil
import soundfile as sf
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

# Скачивание и распаковка датасета
def download_and_extract_librispeech():
    dataset_url = "https://www.openslr.org/resources/12/test-clean.tar.gz"
    dataset_path = "test-clean.tar.gz"
    extract_path = "LibriSpeech"

    # Скачивание датасета (если еще не скачан)
    if not os.path.exists(dataset_path):
        os.system(f"wget {dataset_url} -O {dataset_path}")

    # Распаковка датасета (если еще не распакован)
    if not os.path.exists(extract_path):
        with tarfile.open(dataset_path, "r:gz") as tar:
            tar.extractall(path=".")

# Загрузка данных из LibriSpeech
def load_librispeech_data():
    data = []
    base_path = "LibriSpeech/test-clean"
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.endswith(".txt"):
                with open(os.path.join(root, file), "r") as f:
                    for line in f:
                        parts = line.strip().split(" ", 1)
                        if len(parts) == 2:
                            audio_id, text = parts
                            audio_path = os.path.join(root, f"{audio_id}.flac")
                            if os.path.exists(audio_path):
                                data.append({"audio": audio_path, "text": text})
    return data

# Отделение 60 примеров
def select_samples(data, num_samples=60):
    np.random.seed(42)  # Для воспроизводимости
    indices = np.random.choice(len(data), num_samples, replace=False)
    return [data[i] for i in indices]

# Функция для генерации аудио
def generate_audio(text, description, model, tokenizer, device):
    input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
    prompt_input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
    audio = generation.cpu().numpy().squeeze()
    return audio

Распаковываем 60 записей в директорию ./test

In [31]:
test_dir = "test"

# Скачивание и распаковка датасета
download_and_extract_librispeech()

# Загрузка данных
librispeech_data = load_librispeech_data()

# Отделение 63 примеров
selected_data = select_samples(librispeech_data, num_samples=63)

# Очистка и создание директории 'test'
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
os.makedirs(test_dir)

In [32]:
with open(os.path.join(test_dir, "transcriptions.txt"), "w", encoding="utf-8") as text_file:
    # Обработка каждого примера в тестовой выборке
    for i, example in enumerate(selected_data):
        text = example["text"]
        description = "A young female speaks normal, audio is very clear."  # Описание для генерации

        print(f"Predicting the {i:>5}/{len(selected_data)}")

        # Генерация аудио с помощью TTS
        audio = generate_audio(text, description, model, tokenizer, "cuda" if torch.cuda.is_available() else "cpu")

        # Сохранение сгенерированного аудиофайла
        generated_audio_file = os.path.join(test_dir, f"generated_audio_{i}.wav")
        sf.write(generated_audio_file, audio, model.config.sampling_rate)

        # Извлечение и сохранение исходного аудио
        original_audio, original_sr = sf.read(example["audio"])
        original_audio_file = os.path.join(test_dir, f"original_audio_{i}.wav")
        sf.write(original_audio_file, original_audio, original_sr)

        # Сохранение текста в файл
        text_file.write(f"generated_audio_{i}.wav: {text}\n")
        text_file.write(f"original_audio_{i}.wav: {text}\n")

print("All audio files and transcriptions have been saved.")

Predicting the     0/63
Predicting the     1/63
Predicting the     2/63
Predicting the     3/63
Predicting the     4/63
Predicting the     5/63
Predicting the     6/63
Predicting the     7/63
Predicting the     8/63
Predicting the     9/63
Predicting the    10/63
Predicting the    11/63
Predicting the    12/63
Predicting the    13/63
Predicting the    14/63
Predicting the    15/63
Predicting the    16/63
Predicting the    17/63
Predicting the    18/63
Predicting the    19/63
Predicting the    20/63
Predicting the    21/63
Predicting the    22/63
Predicting the    23/63
Predicting the    24/63
Predicting the    25/63
Predicting the    26/63
Predicting the    27/63
Predicting the    28/63
Predicting the    29/63
Predicting the    30/63
Predicting the    31/63
Predicting the    32/63
Predicting the    33/63
Predicting the    34/63
Predicting the    35/63
Predicting the    36/63
Predicting the    37/63
Predicting the    38/63
Predicting the    39/63
Predicting the    40/63
Predicting the  

# Оценка естественности (UTMOS) на Libri Speech test clean

In [33]:
!python3 compute_utmos.py

Using cache found in /root/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
File: test/generated_audio_0.wav
UTMOS: 4.1716
----------------------------------------------------------------------------------------------------
File: test/generated_audio_1.wav
UTMOS: 3.9030
----------------------------------------------------------------------------------------------------
File: test/generated_audio_2.wav
UTMOS: 4.1809
----------------------------------------------------------------------------------------------------
File: test/generated_audio_3.wav
UTMOS: 4.3384
----------------------------------------------------------------------------------------------------
File: test/generated_audio_4.wav
UTMOS: 4.1807
--------------------------------------------------------------

# Оценка Разборчивости, точность произношения (WER) на Libri Speech test clean

In [34]:
!python3 compute_wer.py

2025-01-07 23:33:31.076237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 23:33:31.098143: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 23:33:31.104291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 23:33:32.504166: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking cha

# Оценка сходства голосов (SECS) на Libri Speech test clean

In [35]:
!python3 compute_secs.py

2025-01-07 23:38:57.106347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 23:38:57.127039: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 23:38:57.133012: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 23:38:58.380572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
File: 0
Original: test/original_audio_0.w

# Итоги

| Датасет \ метрики           | WER    | UTMOS  | SECS   |
|-----------------------------|--------|--------|--------|
| **LibriSpeech Test Clean**  | 0.4946 | 4.1036 | 0.6337 |
| **Jenny**                   | 0.2074 | 4.1682 | 0.9356 |


Бенчмарки на датасете **LibriSpeech Test Clean** получились хуже, что во-первых, возможно из-за того, что аудио из датасета **Jenny** использовалось для трейна, во-вторых: для генерации аудио из датасета **LibriSpeech Test Clean**, возможно, использовался не сильно релевантный промт.